In [17]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tempfile
import urllib
import pandas as pd
import os
from tensorflow.examples.tutorials.mnist import input_data

In [18]:
max_num_checkpoint = 10
num_classes = 2
batch_size = 512
num_epochs = 100

initial_learning_rate = 0.001
learning_rate_decay_factor = 0.95
num_epochs_per_decay = 1

is_training = False
fine_tuning = False
online_test = True
allow_soft_placement = True
log_device_placement = False

In [19]:
mnist = input_data.read_data_sets("MNIST_data/", reshape=True, one_hot=False)

data = {}

data["train/image"] = mnist.train.images
data["train/label"] = mnist.train.labels
data["test/image"] = mnist.test.images
data["test/label"] = mnist.test.labels

def extract_samples_Fn(data):
    index_list = []
    for sample_index in range(data.shape[0]):
        label = data[sample_index]
        if label == 1 or label == 0:
            index_list.append(sample_index)
    return index_list

index_list_train = extract_samples_Fn(data['train/label'])

index_list_test = extract_samples_Fn(data['test/label'])

data['train/image'] = mnist.train.images[index_list_train]
data['train/label'] = mnist.train.labels[index_list_train]

data['test/image'] = mnist.test.images[index_list_test]
data['test/label'] = mnist.test.labels[index_list_test]

dimensionality_train = data['train/image'].shape

num_train_samples = dimensionality_train[0]
num_features = dimensionality_train[1]
print(num_train_samples)
print(num_features)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
11623
784


In [20]:
graph = tf.Graph()
with graph.as_default():
    # global step
    global_step = tf.Variable(0, name="global_step", trainable=False)
    # learning rate policy
    decay_steps = int(num_train_samples / batch_size * num_epochs_per_decay)
    
    learning_rate = tf.train.exponential_decay(initial_learning_rate,
                                              global_step,
                                              decay_steps,
                                              learning_rate_decay_factor,
                                              staircase=True,
                                               name='eponential_decay_learning_rate'
                                              )
    # defining place holders
    image_place = tf.placeholder(tf.float32, shape=([None, num_features]), name='image')
    label_place = tf.placeholder(tf.int32, shape=([None,]),name='gt')
    label_one_hot = tf.one_hot(label_place,depth=num_classes)
    dropout_param = tf.placeholder(tf.float32)
    
    # Model + loss + Accuracy
    # fully connected with two class and softmax 
    logits = tf.contrib.layers.fully_connected(inputs=image_place, num_outputs=num_classes,scope='fc')
    
    # loss
    with tf.name_scope('loss'):
        loss_tensor = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=label_one_hot))
    
    # accuracy  evaluate the model
    prediction_correct = tf.equal(tf.argmax(logits,1), tf.argmax(label_one_hot,1))
    
    # accuracy calculation
    accuracy = tf.reduce_mean(tf.cast(prediction_correct,tf.float32))
    
    # 优化器
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    
    # gradient update
    with tf.name_scope('train_op'):
        gradients_and_variables = optimizer.compute_gradients(loss_tensor)
        train_op = optimizer.apply_gradients(gradients_and_variables,global_step=global_step)
    
    # Run the session
    session_conf = tf.ConfigProto(
        allow_soft_placement=allow_soft_placement,
        log_device_placement=log_device_placement)
    sess = tf.Session(graph=graph,config=session_conf)
    
    with sess.as_default():
        # saver op
        saver = tf.train.Saver()
        # initialize all variables
        sess.run(tf.global_variables_initializer())
        # the prefix for checkpoint files
        checkpoint_prefix = 'model'
        
        if fine_tuning:
            saver.restore(sess, os.path.join(checkpoint_path,checkpoint_prefix))
            print("Model restored for fine-tuning...")
        
        # training  loop over batches
        
        test_accuracy = 0
        for epoch in range(num_epochs):
            total_batch_training = int(data['train/image'].shape[0] / batch_size)
            
            for batch_num in range(total_batch_training):
                start_index = batch_num * batch_size
                end_index = (batch_num + 1) * batch_size
                
                train_batch_data, train_batch_label = data['train/image'][start_index:end_index], data['train/label'][start_index:end_index]
             
                batch_loss, _, training_step = sess.run([loss_tensor,train_op,global_step],
                                                   feed_dict={image_place:train_batch_data,
                                                             label_place:train_batch_label,
                                                             dropout_param:0.5})
            print("Epoch " + str(epoch + 1)+ ", Training Loss = "+ \
                 "{:.5f}".format(batch_loss))
        
        test_accuracy = 100 * sess.run(accuracy, feed_dict={
            image_place:data['test/image'],
            label_place:data['test/label'],
            dropout_param:1.
        })
        print("Final Test Accuracy is %% %.2f" % test_accuracy)

    

Epoch 1, Training Loss = 0.24886
Epoch 2, Training Loss = 0.13265
Epoch 3, Training Loss = 0.08926
Epoch 4, Training Loss = 0.06749
Epoch 5, Training Loss = 0.05434
Epoch 6, Training Loss = 0.04547
Epoch 7, Training Loss = 0.03909
Epoch 8, Training Loss = 0.03430
Epoch 9, Training Loss = 0.03059
Epoch 10, Training Loss = 0.02762
Epoch 11, Training Loss = 0.02521
Epoch 12, Training Loss = 0.02322
Epoch 13, Training Loss = 0.02155
Epoch 14, Training Loss = 0.02013
Epoch 15, Training Loss = 0.01891
Epoch 16, Training Loss = 0.01785
Epoch 17, Training Loss = 0.01692
Epoch 18, Training Loss = 0.01611
Epoch 19, Training Loss = 0.01539
Epoch 20, Training Loss = 0.01475
Epoch 21, Training Loss = 0.01417
Epoch 22, Training Loss = 0.01366
Epoch 23, Training Loss = 0.01319
Epoch 24, Training Loss = 0.01277
Epoch 25, Training Loss = 0.01238
Epoch 26, Training Loss = 0.01203
Epoch 27, Training Loss = 0.01171
Epoch 28, Training Loss = 0.01141
Epoch 29, Training Loss = 0.01114
Epoch 30, Training Loss